In [3]:
import pandas as pd
import tensorflow as tf


In [4]:
data = pd.read_csv("data/winequality_red.csv")

In [5]:
rep_cols = []
for i in data.columns:
    rep_cols.append(i.replace(" ",""))

In [6]:
data.columns = rep_cols

In [7]:
data["y"] = data["quality"]
data = data.drop("quality", axis = 1)

In [8]:
cols = list(data.columns)
label_col = cols[-1]
input_col = cols[:-1]

In [9]:
feature_columns = []
feature_input = {}
for header in input_col:
    feature_columns.append(tf.feature_column.numeric_column(header))
    feature_input[header] = tf.keras.Input(shape=(1,),dtype=tf.float64, name=header)
    
#label = tf.feature_column.categorical_column_with_vocabulary_list(
#      'y', ['3', '4', '5', '6', '7', '8'])
#label_one_hot = tf.feature_column.indicator_column(label)
#feature_input['y'] = tf.keras.Input(shape=(1,),dtype=tf.string,name=header)

In [10]:
data["y"] = data["y"]-3

In [11]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size = 0.3)

In [12]:
train

,fixedacidity,volatileacidity,citricacid,residualsugar,chlorides,freesulfurdioxide,totalsulfurdioxide,density,pH,sulphates,alcohol,y
1283,8.7,0.630,0.28,2.70,0.096,17.0,69.0,0.99734,3.26,0.63,10.2,3
145,8.1,0.670,0.55,1.80,0.117,32.0,141.0,0.99680,3.17,0.62,9.4,2
105,8.1,0.575,0.22,2.10,0.077,12.0,65.0,0.99670,3.29,0.51,9.2,2
351,9.1,0.795,0.00,2.60,0.096,11.0,26.0,0.99940,3.35,0.83,9.4,3
950,8.9,0.120,0.45,1.80,0.075,10.0,21.0,0.99552,3.41,0.76,11.9,4
272,10.9,0.370,0.58,4.00,0.071,17.0,65.0,0.99935,3.22,0.78,10.1,2
343,10.9,0.390,0.47,1.80,0.118,6.0,14.0,0.99820,3.30,0.75,9.8,3
861,5.8,1.010,0.66,2.00,0.039,15.0,88.0,0.99357,3.66,0.60,11.5,3
367,10.4,0.575,0.61,2.60,0.076,11.0,24.0,1.00000,3.16,0.69,9.0,2
282,7.6,0.520,0.12,3.00,0.067,12.0,53.0,0.99710,3.36,0.57,9.1,2


In [13]:
def data_to_tensor(df,batch_size=32,shuffle=True):
    df = df.copy()
    data_y = tf.one_hot(df["y"],6)
    data_x = df.drop("y", axis=1)
    ts_data = tf.data.Dataset.from_tensor_slices((dict(data_x),data_y))
    if shuffle:
        ts_data = ts_data.shuffle(buffer_size=len(data))
    ts_data = ts_data.batch(batch_size)
    return ts_data

In [14]:
train_ds = data_to_tensor(train)
test_ds = data_to_tensor(test, shuffle=False)

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(feature_columns=feature_columns),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

In [16]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
model.fit(train_ds, validation_data = test_ds, epochs=10)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 35 steps, validate on 15 steps
Epoch 1/10
35/35 [==============================] - 0s 8ms/step - loss: 1.3735 - acc: 0.4298 - val_loss: 1.0926 - val_acc: 0.5188
Epoch 2/10
35/35 [==============================] - 0s 2ms/step - loss: 1.1810 - acc: 0.4987 - val_loss: 1.1043 - val_acc: 0.5208
Epoch 3/10
35/35 [==============================] - 0s 2ms/step - loss: 1.2086 - acc: 0.4924 - val_loss: 1.1465 - val_acc: 0.4708
Epoch 4/10
35/35 [==============================] - 0s 2ms/step - loss: 1.1834 - acc: 0.4933 - val_loss: 1.1365 - val_acc: 0.4812
Epoch 5/10
35/35 [==============================] - 0s 2ms/step - loss: 1.1400 - acc: 0.5228 - val_loss: 1.1534 - val_acc: 0.4667
Epoch 6/10
35/35 [==============================] - 0s 2ms/step - loss: 1.1409 - acc: 0.4853 - val_loss: 1.0757 - val_acc: 0.5042
Epoch 7/10
35/35 [==============================] - 0s 2ms/step - loss: 1.1267 - acc: 0.5022 - val_

In [18]:
from witwidget.notebook.visualization import WitConfigBuilder
from witwidget.notebook.visualization import WitWidget


In [19]:
import numpy as np
def df_to_examples(df, columns=None):
    examples = []
    if columns == None:
        columns = df.columns.values.tolist()
    for index, row in df.iterrows():
        example = tf.train.Example()
        for col in columns:
            if df[col].dtype is np.dtype(np.int64):
                example.features.feature[col].int64_list.value.append(int(row[col]))
            elif df[col].dtype is np.dtype(np.float64):
                example.features.feature[col].float_list.value.append(row[col])
            elif row[col] == row[col]:
                example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
        examples.append(example)
    return examples

In [20]:
example = df_to_examples(data)

In [26]:
config_builder = WitConfigBuilder(example).set_estimator_and_feature_spec(
    model, feature_input).set_compare_estimator_and_feature_spec(
    model, feature_input).set_model_type('classification')

In [27]:
a = WitWidget(config_builder, height=1000)